In [7]:
from Regression import Regression
from Franke import FrankeNoise
import numpy as np

In [8]:
reg = Regression()
x, y = np.linspace(0, 1, 101), np.linspace(0, 1, 201)
reg.generate_generic_data(x, y, FrankeNoise)

asdf (101,) (201,)
(201, 101) (201, 101)
f: (201, 101)
